In [1]:
from helper import read_data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import KFold
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn.cross_validation import KFold
from sklearn import preprocessing

%matplotlib inline

type(read_data)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


function

In [2]:
RANDOM_SEED = 9
TRAIN_FILE_PATH = "./asset/training_data.txt"

VOWELS_TABLE = """\
|   AA  |   ɑ   |
|   AE  |   æ   |
|   AH  |   ʌ   |
|   AO  |   ɔ   |
|   AW  |   aʊ  |
|   AY  |   aɪ  |
|   EH  |   ɛ   |
|   ER  |   ɜːr |
|   EY  |   eɪ  |
|   IH  |   ɪ   |
|   IY  |   i   |
|   OW  |   oʊ  |
|   OY  |   ɔɪ  |
|   UH  |   ʊ   |
|   UW  |   u   |
"""

CONSONANT_TABLE = """\
|     P     |  p  |
|     S     |  s  |
|     B     |  b  |
|     SH    |  ʃ  |
|     CH    |  tʃ |
|     T     |  t  |
|     D     |  d  |
|     TH    |  θ  |
|     DH    |  ð  |
|     V     |  v  |
|     F     |  f  |
|     W     |  w  |
|     G     |  g  |
|     Y     |  j  |
|     HH    |  h  |
|     Z     |  z  |
|     JH    |  dʒ |
|     ZH    |  ʒ  |
|     K     |  k  |
|     L     |  l  |
|     M     |  m  |
|     N     |  n  |
|     NG    |  ŋ  |
|     R     |  r  |
"""


In [3]:
def init_phonemes(phonemes_table):
    ans = {}
    phonemes_list = list(filter(lambda x: not x in "\n",
                              phonemes_table.replace(" ", "").split("|")))
    for x in range(0, len(phonemes_list), 2):
        ans[phonemes_list[x]] =  phonemes_list[x+1]
    return ans

train_data = read_data(TRAIN_FILE_PATH)

vowels_dict = init_phonemes(VOWELS_TABLE)
consonants_dict = init_phonemes(CONSONANT_TABLE)

In [4]:
def contain_phonemes(phoneme, phonemes_dict):
    return phoneme in phonemes_dict

def count_contain_phonemes_num(phonemes_list, phonemes_dict):
    return len(list(filter(lambda phoneme:contain_phonemes(phoneme, phonemes_dict), phonemes_list)))


def get_world(line):
    return line.split(":")[0]

def get_phonemes(line):
    return line.split(":")[1]


def has_number(string):
    return any(char.isdigit() for char in string)

def get_stress_index(vowels_list):
    return [i + 1 for i, x in enumerate(vowels_list) if "1" in x][0]

# One Way

In [5]:
def get_data_1(line):
    line = get_phonemes(line)
    phonemes_list = line.split(" ")
    vowels_list = list(filter(lambda phoneme: has_number(phoneme), phonemes_list))
    #  return [get_stress_index(vowels_list), line.replace("0", "").replace("1", "").replace("2", "")]
    for i in range(len(phonemes_list)):
        phonemes_list[i] = str(i) + phonemes_list[i].replace("0", "").replace("1", "").replace("2", "") + (str(vowels_list.index(phonemes_list[i])) if phonemes_list[i] in vowels_list else "") 
    return [get_stress_index(vowels_list), " ".join(phonemes_list)]

df = pd.DataFrame([get_data_1(line) for line in train_data], 
                  index=[get_world(line) for line in train_data],
                  columns=["type", "text"])
df.head()

,type,text
COED,1,0K 1OW0 2EH1 3D
PURVIEW,1,0P 1ER0 2V 3Y 4UW1
HEHIR,1,0HH 1EH0 2HH 3IH1 4R
MUSCLING,1,0M 1AH0 2S 3AH1 4L 5IH2 6NG
NONPOISONOUS,2,0N 1AA0 2N 3P 4OY1 5Z 6AH2 7N 8AH2 9S


In [6]:
def get_freq_of_tokens(sms):
    tokens = {}
    for token in sms.split(' '):
        if token not in tokens:
            tokens[token] = 1
        else:
            tokens[token] += 1
    return tokens

# print(get_freq_of_tokens(df.iloc[0].text))

# features_and_labels = []
# for index in range(len(df)):
#     features_and_labels.append((get_freq_of_tokens(df.iloc[index].text), df.iloc[index].type))

    
# encoder = LabelEncoder()
# vectorizer = DictVectorizer(dtype=float, sparse=True)
# x, y = list(zip(*features_and_labels))

# x = vectorizer.fit_transform(x)
# y = encoder.fit_transform(y)


# arr = x[0].toarray()
# for i in range(len(arr[0])):
#     if arr[0][i] > 0:
#         print('{}:{}'.format(vectorizer.feature_names_[i], arr[0][i]))
        


In [9]:
def test_nb(train, test, log=True):
    x_train = [ get_freq_of_tokens(text) for text in  train.text ]
    y_train = train.type
    x_test = [ get_freq_of_tokens(text) for text in  test.text ]
    y_test = test.type
    
    encoder = LabelEncoder()
    vectorizer = DictVectorizer(dtype=float, sparse=True)
    
    x = vectorizer.fit_transform(x_train)
    y = encoder.fit_transform(y_train)
    
    nb = MultinomialNB(alpha=1)
    nb.fit(x, y)
    train_err = nb.score(x, y)
    
    x = vectorizer.transform(x_test)
    y = encoder.transform(y_test)
    test_err = nb.score(x, y)
    
    if log:
        print("{}, {}, {}".format(train_err, test_err, train_err - test_err))
    return train_err, test_err
 
def evalute_nb(times=10, log=True):
    total_train_err, total_test_err = 0, 0
    for i in range(times):
        train_err, test_err = test_nb(*train_test_split(df, test_size=0.2, random_state=RANDOM_SEED+i))
        total_train_err = total_train_err + train_err
        total_test_err  = total_test_err  + test_err
    print("avg:")
    print("{}, {}, {}".format(total_train_err / times, total_test_err / times, (total_train_err - total_test_err) / times))

evalute_nb()

0.869525, 0.8693, 0.00022500000000003073
0.870125, 0.868, 0.0021250000000000435
0.870925, 0.8642, 0.006724999999999981
0.870825, 0.8642, 0.006624999999999992
0.8699, 0.8692, 0.0007000000000000339
0.8694, 0.8651, 0.0042999999999999705
0.8695, 0.8686, 0.0009000000000000119
0.870975, 0.8671, 0.003875000000000073
0.87015, 0.8664, 0.003750000000000031
0.870325, 0.8678, 0.0025249999999999995
avg:
0.8701650000000001, 0.8669899999999998, 0.003175000000000239


In [8]:
nb = BernoulliNB(alpha=1, binarize=0.0) # every value >0.0 will be binarized to 1
nb.fit(x, y)
nb.score(x, y)

NameError: name 'x' is not defined

# Another Way

In [ ]:
def get_data(line):
    line = get_phonemes(line)
    phonemes_list = line.split(" ")
    vowels_list = list(filter(lambda phoneme: has_number(phoneme), phonemes_list))
    consonants_list = list(filter(lambda phoneme: not has_number(phoneme), phonemes_list))
    # print(count_contain_phonemes_num(constants_list, consonants_dict))
    
    vowels_vector = list(vowels_dict)
    for i in range(len(vowels_vector)):
        vowels_vector[i] = 1 if vowels_vector[i] in [vowel[0:2] for vowel in vowels_list] else 0
        
    consonants_vector = list(consonants_dict)
    for i in range(len(consonants_vector)):
        consonants_vector[i] = 1 if consonants_vector[i] in consonants_list else 0
        
    return [get_stress_index(vowels_list), len(vowels_list), len(consonants_list)] + vowels_vector + consonants_vector

In [ ]:
df = pd.DataFrame([get_data(line) for line in train_data], 
                  index=[get_world(line) for line in train_data],
                  columns=["stress_index", "vowels_size", "constans_size"] + list(vowels_dict) + list(consonants_dict))
df.head()
# df.describe()

In [ ]:
# df[(df["vowels_size"] == 1) | (df["vowels_size"] == 5)].size

# df.groupby(['vowels_size', 'stress_index']).agg(
#     {'constans_size':'count'}).groupby(level=0).apply(lambda x : 
# x * 100 / float(x.sum())).add_suffix('_Count').reset_index().columns[1]

# df.groupby(['vowels_size', 'stress_index']).agg(
#     {'constans_size':'count'}).groupby(level=0).apply(lambda x :                          
#                                                       x * 100 / float(x.sum())).groupby("vowels_size").plot.pie(subplots=True)

feature_list = list(df.corr().stress_index.to_frame().sort_values(by='stress_index', ascending=False).index)[1:]

## Decision Tree

In [ ]:
def experiment_once(train, test, feature_list, depth=5, log=False):
    x_train = train[feature_list]
    x_test = test[feature_list]
    y_train = train.stress_index
    y_test = test.stress_index

    clf = DecisionTreeClassifier(criterion = "gini", max_depth=depth, random_state=RANDOM_SEED)
    dtree = clf.fit(x_train, y_train)
    train_err = dtree.score(x_train, y_train)
    test_err = dtree.score(x_test, y_test)
    if log:
        print("{}, {}, {}".format(train_err, test_err, train_err - test_err))
    return train_err, test_err

def evalute(feature_list, times=10, depth=5, log=False):
    total_train_err, total_test_err = 0, 0
    for i in range(times):
        train, test = train_test_split(df, test_size=0.2, random_state=RANDOM_SEED + i)
        train_err, test_err = experiment_once(train, test, feature_list, depth, log)
        total_train_err = total_train_err + train_err
        total_test_err  = total_test_err  + test_err
    print("avg:")
    print("{}, {}, {}".format(total_train_err / times, total_test_err / times, (total_train_err - total_test_err) / times))

evalute(feature_list[:10], 10, depth=10, log=True)

## (Gaussian) Naive Bayes Classifier

In [ ]:
x = df[feature_list]
y = df.stress_index
gnb = GaussianNB()
gnb.fit(x, y)
gnb.score(x, y)

## KNN Classifier

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
neigh = KNeighborsClassifier(n_neighbors=3,weights='uniform',p=2)
neigh.fit(x_scaled, y)
neigh.score(x_scaled, y)